In [1]:
# install missing packages
%pip install plotly
%pip install py-cpuinfo
%pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# check system details
import os
import psutil
import cpuinfo

ram_info = psutil.virtual_memory()
print(f"Total RAM: {ram_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available RAM: {ram_info.available / 1024 / 1024 / 1024:.2f} GB")
print(f"Used RAM: {ram_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of RAM: {ram_info.percent}%")
print(f"CPU Cores: {os.cpu_count()}")
print(f"CPU Speed: {cpuinfo.get_cpu_info()['hz_actual_friendly']}")
disk_info = psutil.disk_usage(os.getcwd())
print(f"Total Disk: {disk_info.total / 1024 / 1024 / 1024:.2f} GB")
print(f"Available Disk: {disk_info.free / 1024 / 1024 / 1024:.2f} GB")
print(f"Used Disk: {disk_info.used / 1024 / 1024 / 1024:.2f} GB")
print(f"Percentage Usage Of Disk: {disk_info.percent}%")

Total RAM: 15.47 GB
Available RAM: 14.08 GB
Used RAM: 1.12 GB
Percentage Usage Of RAM: 9.0%
CPU Cores: 4
CPU Speed: 2.5000 GHz
Total Disk: 24.99 GB
Available Disk: 16.77 GB
Used Disk: 8.22 GB
Percentage Usage Of Disk: 32.9%


In [3]:
# import requirements
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from gbm_regressor import Regression

In [4]:
# get the data
riders = pd.read_csv("riders.csv")
riders = riders.sort_values(by=["date", "route", "daytype"]).reset_index(drop=True)
riders = riders.head(int(0.05 * riders.shape[0]))  # only use 5% of the data

In [5]:
# resort the data
riders = riders.sort_values(by=["route", "daytype", "date"]).reset_index(drop=True)

In [6]:
# lag the output variable
riders["rides(t-1)"] = riders.groupby(["route", "daytype"])["rides"].shift(1)
riders["rides(t-2)"] = riders.groupby(["route", "daytype"])["rides"].shift(2)
riders["rides(t-3)"] = riders.groupby(["route", "daytype"])["rides"].shift(3)
riders["rides(t-4)"] = riders.groupby(["route", "daytype"])["rides"].shift(4)

In [7]:
# remove rows with missing values
riders = riders.dropna().reset_index(drop=True)

In [8]:
# resort the data
riders = riders.sort_values(by=["date", "route", "daytype"]).reset_index(drop=True)

In [9]:
# split up the data into training and testing
y = riders[["rides"]]
X = riders.drop(columns="rides")
trainX = X.head(int(0.8 * X.shape[0]))
trainy = y.head(int(0.8 * y.shape[0]))
testX = X.tail(int(0.2 * X.shape[0])).reset_index(drop=True)
testy = y.tail(int(0.2 * y.shape[0])).reset_index(drop=True)

In [10]:
# build the model
print("\n---- Riders Regression Analysis ----\n")
model = Regression(
    name="XGBoost With Feature Engineering", 
    frac=1,
    rename=True, 
    time=True, 
    binary=True, 
    imputation=False, 
    variance=True,
    scale=True,
    atwood=True,
    binning=True,
    reciprocal=True, 
    interaction=True, 
    selection=True,
)
try:
    model.load()  # load the machine learning pipeline
    predictions = model.predict(testX)
except:
    model.fit(trainX, trainy)  # build the machine learning pipeline
    predictions = model.predict(testX)


---- Riders Regression Analysis ----

1/6) Model Training
> Renaming Features
> Extracting Time Features
> Transforming Categorical Features
> Renaming Features
> Removing Constant Features
> Scaling Features
> Selecting Features
> Computing Atwoood Numbers
> Binning Features
> Computing Reciprocals
> Computing Interactions
> Removing Constant Features
> Selecting Features
> Transforming The Training Data
> Training XGBoost
4.23 Minutes
2/6) Model Performance
> Transforming The Testing Data
7.45 Seconds
3/6) Model Deployment
> Transforming All The Data
> Training XGBoost
1.47 Minutes
4/6) Model Indicators
0.13 Seconds
5/6) Model Prediction
> Transforming The New Data
4.03 Seconds
6/6) Model Monitoring
12.15 Seconds


In [11]:
# model diagnostics
print("Model Diagnostics:")
print(f"> RMSE: {model.rmse}")
print(f"> R2: {model.r2}")
print(f"> In Control: {model.in_control}")
print("> Model Indicators:")
for i, indicator in enumerate(model.indicators["Indicator"][:10]):
    print(f"{i+1}. {indicator}")
print("> Feature Drift:")
for i, feature in enumerate(model.drift.loc[model.drift["pvalue"] < 0.05, "Feature"][:10]):
    print(f"{i+1}. {feature}")
if model.drift.loc[model.drift["pvalue"] < 0.05].shape[0] == 0:
    print("None")

Model Diagnostics:
> RMSE: 1451.663951192435
> R2: 0.9381333405127853
> In Control: 94.45%
> Model Indicators:
1. rides(t-3)(0.2-0.4)*1/rides(t-1)
2. rides(t-4)(0.2-0.4)*1/rides(t-1)
3. rides(t-1)*date_nasdaq(0.2-0.4)
4. rides(t-4)*date_gdi(0.2-0.4)
5. rides(t-1)(0.6-0.8)*1/date_gdi
6. rides(t-1)
7. 1/rides(t-1)*1/rides(t-3)
8. rides(t-1)(0.6-0.8)*1/date_nasdaq
9. date_gdp
10. rides(t-4)*date_nasdaq(0.2-0.4)
> Feature Drift:
1. rides(t-2)*date_day_of_week_Monday
2. rides(t-1)*date_federal_funds_rate(0.6-0.8)
3. rides(t-3)(0.4-0.6)*1/rides(t-4)
4. date_ppi*rides(t-3)(0.4-0.6)
5. rides(t-3)(0.4-0.6)
6. rides(t-3)(0.4-0.6)*1/rides(t-1)
7. rides(t-1)*rides(t-2)(0.4-0.6)
8. rides(t-2)(0.4-0.6)*1/rides(t-3)
9. rides(t-3)*rides(t-2)(0.4-0.6)
10. rides(t-1)*rides(t-3)(0.4-0.6)


In [12]:
# score the model
rmse = mean_squared_error(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
    squared=False,
)
r2 = r2_score(
    y_true=testy.iloc[:,0].to_numpy(),
    y_pred=predictions,
)

print(f"RMSE: {rmse}")
print(f"R2: {r2}")

RMSE: 1639.5242328377822
R2: 0.9243763357908555


In [13]:
# save the machine learning pipeline
model.dump()

In [14]:
# refit the model to include the test data
model.refit(testX, testy)

Model Retraining:
> Transforming The New Data
> Training XGBoost
1.61 Minutes
